# Analiza głównych składowyn (PCA)

# Principal component analysis (PCA)

In [ ]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt

Mamy zestaw danych będący 40 punktami na płaszczyźnie $(x_i,y_i).$

We have a dataset that is 40 points on the $(x_i,y_i).$ plane

In [ ]:
X = np.empty((40,2))
X[:,0] = np.linspace(-1,1,40)+ np.random.randn(40)*0.1
X[:,1] = 1.1*X[:,0] + np.random.randn(40)*0.2
print(X)

## Wstęp


## Introduction

### Macierz kowariancji

  $$ C_{ij} = \frac{1}{N-1}\sum_{k=1}^N{ X_{ki} X_{kj}}$$

### Covariance matrix

  $$C_{ij}=\frac{1}{N-1}\sum_{k=1}^N{ X_{ki} X_{kj}}$$

###  Oblicz macierz kowariancji z danych zawartych w  `X` korzystając z funkcji `np.cov`.

### Calculate the covariance matrix from the data contained in `X` using the function` e.g. cov`.

In [ ]:
### BEGIN SOLUTION
C = np.cov(X.T)
### END SOLUTION
C

In [ ]:
assert C.shape == (2,2)


###  Oblicz macierz kowariancji z danych zawartych w  `X` korzystając z funkcji `np.mean` `np.sum`

### Calculate the covariance matrix from the data contained in `X` using the function` e.g. mean` e.g. e.g. sum`

In [ ]:
def kowariancja(X):
### BEGIN SOLUTION
    m = np.mean(X,axis=0)
    C = 1/(X.shape[0]-1)*np.dot( (X-m).T,X-m)
    return C
### END SOLUTION

In [ ]:
np.testing.assert_almost_equal(kowariancja(X), np.cov(X.T))

import inspect
blacklist = ["cov","corrcoef"]
assert all([ not keyword  in inspect.getsource(kowariancja) for keyword in blacklist])

### Wartości i wektory własne

Narysujmy dane


### Eigenvalues ​​and eigenvectors

Let's draw the data

In [ ]:
plt.figure()
plt.axes().set_aspect('equal', 'datalim')
plt.scatter(X[:,0],X[:,1])

### Zagadnienie własne

Szukamy takich liczb $\lambda_i$ i wektorów $x_i$ by zachodziło:
$$ C x_i = \lambda_i x_i $$


W numpy mamy gotową funkcję obliczającą liczby $\lambda_i$ i wektory $x_i$:

### Eigenproblem

We are looking for such $\lambda_i$ numbers and $x_i$ vectors to occur:
$$ C x_i = \lambda_i x_i $$


In numpy we have a ready function that calculates $\lambda_i$ numbers and $x_i$ vectors:

In [ ]:
val, ev =  np.linalg.eig(C)


### Sprawdźmy rachunkiem równanie własne:

Niech `lam` będzie wartością własną a `vec` wektorem:

### Let's check the eigenvalue problem:

Let `lam` be an eigenvalue and` vec` be a vector:

In [ ]:
lam = None
vec = None

### BEGIN SOLUTION
lam = val[1]
vec = ev[:,1]
### END SOLUTION

To zachodzi:
    
`np.dot(C,vec) == lam*vec`

It is fulfilled:
    
`e.g..dot (C, vec) == lam * vec`

In [ ]:
np.testing.assert_allclose(np.dot(C,vec), lam*vec)
assert abs(lam)>1e-6

Dodajmy na rysunku wektory własne:

Let's add our eigenvectors in the plot:

In [ ]:
plt.figure()
plt.axes().set_aspect('equal', 'datalim')
plt.scatter(X[:,0],X[:,1])

plt.arrow(0,0,ev[0,0],ev[1,0],color='r')
plt.arrow(0,0,ev[0,1],ev[1,1],color='b')

Sprawdźmy jeszcze,  że wektory własne odpowiadające różnym wartościom własnym są ortogonalne.
    
    

Let's check that the eigenvectors corresponding to different eigenvalues ​​are orthogonal.

In [ ]:
np.testing.assert_approx_equal( np.dot(ev[:,0],ev[:,0]), 1)
np.testing.assert_approx_equal( np.dot(ev[:,0],ev[:,1]), 0)

## Implementacja PCA krok po kroku

Wykonamy redukcję PCA 13 wymiarowego  zestawu do przestrzeni 3d


Referencyjną implementacją będzie `sklearn.decomposition.PCA`.

Niech:

  - `Xorig`  - dane orginalne (przed PCA)
  - `X` - dane w zredukowanej przestrzeni

Zaimplementujemy taką samą operację konstruując operator rzutowania z 13 do 3 wymiarów z trzech wektorów własnych macierzy kowariancji. Wektory wybierzemy kierując się tym by odpowiadały one największym wartościom własnym.



## PCA - step by step implementation

We will reduce the PCA 13 dimensional kit to 3d space


The reference implementation will be `sklearn.decomposition.PCA`.

Let:

  - `Xorig` - original data (before PCA)
  - `X` - data in a reduced space

We will implement the same operation by constructing the projection operator from 13 to 3 dimensions from three eigenvectors of the covariance matrix. We will choose the vectors guided by the fact that they correspond to the highest eigenvalues.

In [ ]:
from sklearn import decomposition
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

X,y  = datasets.load_wine(return_X_y=True)
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

pca = decomposition.PCA(n_components=3)
pca.fit(X)

Xorig = X.copy()
X = pca.transform(X)

In [ ]:
Xorig.shape,"->", X.shape

### Krok 1: oblicz macierz kowariancji

### Step 1: calculate the covariance matrix

In [ ]:
C = None
### BEGIN SOLUTION
C = np.cov(Xorig.T)
### END SOLUTION

In [ ]:
np.testing.assert_approx_equal(C[11,2],0.0039333279)
np.testing.assert_almost_equal(C,np.cov(Xorig.T))

### Krok 2:  Wektory i wartości własne macierzy kowariancji. 

Rozwiązujemy numerycznie zagadnienie własne macierzy $C$:

### Step 2: Vectors and eigenvalues ​​of the covariance matrix.

We numerically solve the own problem of $C$ matrix:

In [ ]:
val, ev =  np.linalg.eig(C)

**Posortuj** wartości i odpowiadające im wektory własne od największej do najmniejszej wartości własnej.

*warto rozważyć zastosowanie* `np.argsort`.

**Sort** values ​​and their corresponding eigenvectors from highest to smallest eigenvalues.

*it is worth considering the use of* `e.g.argsort`.

In [ ]:
### BEGIN SOLUTION
sort_idx = np.argsort(val)[::-1]
val = val[sort_idx]
ev = ev[:,sort_idx]
### END SOLUTION

Pierwsze trzy wartości własne powinny być takie same jak liczby w atrybucie `pca.explained_variance_`

In [ ]:
np.testing.assert_allclose(val[-1],0.1039619918207, rtol=1e-5)
np.testing.assert_allclose(pca.explained_variance_, val[:3], rtol=1e-5)

### Krok 3: Stwórz macierz operatora rzutowania z przestrzeni 13d w  3d

Mając jeden znormalizowany wektor własny (13 liczb), iloczyn skalarny tego wektora przez dowolny inny wektor tej samej przestrzeni (13d) można zinterpretować jako rzutowanie. Chcemy rzutować na pierwsze trzy wektory własne - można więc przedstawić to jako operację liniową z macierzą $(13,3)$


Macierz taka składa się z trzech 13 elementowych kolumn, będących wartościami własnymi macierzy kowariancji.

### Step 3: Create a projection operator matrix from space 13d in 3d

Having one normalized eigenvector (13 numbers), the scalar product of this vector by any other vector of the same space (13d) can be interpreted as projection. We want to project the first three eigenvectors - so you can represent it as a linear operation with the $(13,3)$ matrix


This matrix consists of three 13-element columns, which are eigenvalues ​​of the covariance matrix.

In [ ]:
P = None 
### BEGIN SOLUTION
P = ev[:,:3]
### END SOLUTION

In [ ]:
P.shape

W sklearn mamy taką macierz w atrybucie `pca.components_`:

In sklearn we have this matrix in the attribute `pca.components_`:

In [ ]:
pca.components_

Sprawdźmy czy otrzymamy te same wyniki:

Let's check if we get the same results:

In [ ]:
np.testing.assert_almost_equal(np.abs(P.T),np.abs(pca.components_))

### Krok 4: Wykonaj rzutowanie wszytkich  danych z 13d do 3d.

Najpierw dla sprawdzenia wykonamy rzutowanie jeden punkt z 13d do 3d (np. z indeksem  12)

### Step 4: Perform projection of all data from 13d to 3d.

First, for checking we will project one point from 13d to 3d (e.g. with index 12)

In [ ]:
print(Xorig[12])

In [ ]:
x12 = None 

### BEGIN SOLUTION
x12 = np.dot(Xorig[12],P)
### END SOLUTION

In [ ]:
print(x12)

In [ ]:
x12.shape = (3,)
np.testing.assert_almost_equal(np.abs(x12), np.abs([-2.11346234,  0.67570634, -0.86508643]))

Możemy teraz wykonać rzutowanie dla wszystkich danych. 

We can now cast to all data.

In [ ]:
X_3d= None

### BEGIN SOLUTION
X_3d = np.dot(Xorig,P)
### END SOLUTION

In [ ]:
X_3d.shape  == (Xorig.shape[0],3)

np.testing.assert_allclose(np.abs(X_3d[:4]),np.abs([[-3.31675081e+00,  1.44346263e+00, -1.65739045e-01],
       [-2.20946492e+00, -3.33392887e-01, -2.02645737e+00],
       [-2.51674015e+00,  1.03115130e+00,  9.82818670e-01],
       [-3.75706561e+00,  2.75637191e+00, -1.76191842e-01]]), rtol=1e-5 )

Ostatecznie wektor w zredukowanej przestrzeni obliczony przez nas i wektor obliczony przez sklear są takie same.

Ultimately, the vector in the reduced space calculated by us and the vector calculated by sklear are the same.

In [ ]:
np.testing.assert_allclose(np.abs(X_3d),np.abs(X), rtol=1e-5)